# 1. Data Processing

This notebook covers the data processing pipeline for the iFood case study.

## 1.1 Setup

Import necessary libraries and configure environment.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.set_option('display.max_columns', None)
sns.set_theme(style=None) 
plt.style.use("fivethirtyeight")
sns.set_style("whitegrid")

## 1.2 Load Data

Load the raw data from the data/raw directory.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("test").getOrCreate()
path = "/Workspace/Users/castrobeneyto@gmail.com/ifood-case/data/raw"

### 1.2.1 Offers

In [0]:
offers_df = spark.read.json(f"{path}/offers.json")
print("Shape: ", (offers_df.count(), len(offers_df.columns)))
offers_df.show()

In [0]:
offers_df.printSchema()

In [0]:
df_exploded = offers_df.select("id", F.explode("channels").alias("channel"))
df_dummies = (
    df_exploded.groupBy("id")
    .pivot("channel")
    .agg(F.lit(1))
    .na.fill(0)
)

# Create dummy variables for the 'channels' column
offers_df = offers_df.join(df_dummies, on="id", how="left").drop("channels")
print("Shape: ", (offers_df.count(), len(offers_df.columns)))
offers_df.show()

In [0]:
offers_df.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in offers_df.columns
]).show()

**Conclusions:**

There are three distinct offer types represented in the dataset:

- `bogo`

- `discount`

- `informational`

BOGO and discount offers include economic conditions, such as:

- `min_value` (minimum spending required)

- `discount_value` (reward/discount amount)

Informational offers, on the other hand, always have zero for both fields.

All offers include communication channels:
- email, mobile, social, web

Most offers are sent through multiple channels simultaneously.

### 1.2.2 Profile

In [0]:
profile_df = spark.read.json(f"{path}/profile.json")
print("Shape: ", (profile_df.count(), len(profile_df.columns)))
profile_df.show(10)

In [0]:
profile_df.printSchema()

In [0]:
profile_df.select([
    F.sum(F.col(c).isNull().cast("int")).alias(c)
    for c in profile_df.columns
]).show()

In [0]:
null_rows = profile_df.filter(
    " OR ".join([f"{c} IS NULL" for c in profile_df.columns])
)
null_rows.select(
    F.min("age").alias("min_age"),
    F.max("age").alias("max_age")
).show()

In [0]:
profile_df = profile_df.withColumn(
    "registered_on",
    F.to_date(F.col("registered_on").cast("string"), "yyyyMMdd")
)
profile_df = profile_df.dropna(subset=['gender', 'credit_card_limit'])
print("Shape: ", (profile_df.count(), len(profile_df.columns)))
profile_df.show(10)

In [0]:
profile_df.groupBy("gender").count().orderBy("count", ascending=False).show()

**Conclusions:**
- The dataset contains several rows where `gender` and `credit_card_limit` are missing.

- All rows with missing values in these two columns correspond to users with an **age of 118**, which strongly suggests these are placeholder or invalid ages rather than real values.

- Because of this, all missing values were removed from the dataset as part of data cleaning.

- After this filtering step, the dataset was reduced from approximately 17,000 users to 14,825, ensuring higher data quality for subsequent analysis.

### 1.2.3 Transactions

In [0]:
transactions_df = spark.read.json(f"{path}/transactions.json")
print("Shape: ", (transactions_df.count(), len(transactions_df.columns)))
transactions_df.show(10)

In [0]:
transactions_df.head()['value']

In [0]:
transactions_df.groupBy("event").count().orderBy("count", ascending=False).show()

In [0]:
transactions_df.select(
    F.min("time_since_test_start").alias("min_age"),
    F.max("time_since_test_start").alias("max_age")
).show()

In [0]:
transactions_df = transactions_df.select(
    "*",        
    F.col("value.*")   
).drop("value")
print("Shape: ", (transactions_df.count(), len(transactions_df.columns)))
transactions_df.show(10)

In [0]:
transactions_df.filter(transactions_df.account_id == "0414fcebbd7f4a19bca352622b1c84e2").show()

In [0]:
transactions_df = transactions_df.withColumn(
    "offer_id_unified",
    F.coalesce(F.col("offer id"), F.col("offer_id"))
).drop("offer id", "offer_id")
print("Shape: ", (transactions_df.count(), len(transactions_df.columns)))
transactions_df.filter(transactions_df.account_id == "0414fcebbd7f4a19bca352622b1c84e2").show()

**Conclusions:**
- The dataset contains four main events: transactions, offers received, viewed, and completed.

- Transaction activity is the largest portion of the dataset.

- Offer-related events correctly have NULL for amount, since they aren't purchases.

- Customers receive multiple offers, suggesting broad distribution rather than targeted campaigns.

- There is an inconsistency between the `offer id` and `offer_id` columns, indicating the need for data cleaning. The `offer id` column is populated for offer received and offer viewed events, whereas `offer_id` is populated only for offer completed events. For clarity in the analysis, both fields were consolidated into a single column, `offer_id_unified`.

## 1.3 Saving Pre-Processed Files

In [0]:
path = "/Workspace/Users/castrobeneyto@gmail.com/ifood-case/data/processed"
offers_df.toPandas().to_csv(f"{path}/offers.csv", index=False)
profile_df.toPandas().to_csv(f"{path}/profile.csv", index=False)
transactions_df.toPandas().to_csv(f"{path}/transactions.csv", index=False)

## 1.4 Exploratory Data Analysis

Initial exploration of the dataset.

In [0]:
offers_df = pd.read_csv(f"{path}/offers.csv")
profile_df = pd.read_csv(f"{path}/profile.csv")
transactions_df = pd.read_csv(f"{path}/transactions.csv")

### 1.4.1 Offers

In [0]:
offers_df.sort_values(by='offer_type')

In [0]:
offers_df['offer_type'].value_counts().sort_index(ascending=True).plot.bar()
plt.show()

In [0]:
offers_df.groupby('offer_type').describe()

**Conclusions:**

- `bogo` and `discount` are the most common offer types in the dataset.

- `bogo` offers present the highest average discount value.

- `discount` offers have the longest average duration, while `informational` offers have the shortest.

- `discount` offers also require the highest minimum spend to qualify for the reward.

### 1.4.2 Profile

In [0]:
profile_df.info()

In [0]:
profile_df.describe()

In [0]:

profile_df["registered_on"] = pd.to_datetime(profile_df["registered_on"])
profile_df.groupby(profile_df['registered_on'].dt.to_period('M'))['id'].count().plot.line()
plt.show()

In [0]:
profile_df['gender'].value_counts().sort_index(ascending=True).plot.bar()
plt.show()

In [0]:
profile_df['age'].hist(bins=20)
plt.show()

In [0]:
profile_df['credit_card_limit'].hist(bins=20)
plt.show()

In [0]:
sns.boxplot(data=profile_df, x="gender", y="credit_card_limit", width=0.6)
plt.show()

In [0]:
sns.boxplot(data=profile_df, x="gender", y="age", width=0.6)
plt.show()

In [0]:
plt.scatter(profile_df["age"], profile_df["credit_card_limit"], alpha=0.3, s=8)
plt.show()

In [0]:
correlation = profile_df[["age", "credit_card_limit"]].corr()
print(correlation)

**Conclusions:**

- The highest volume of registered customers occurred at the end of 2017.

- _Males_ are the most represented group in the dataset, while the _Other_ gender category is notably underrepresented.

- The average consumer age is 54 years.

- `credit_card_limits` show a positively skewed distribution.

- There is no clear visual difference in credit card limits across genders, though male limits appear slightly lower.

- `age` and credit `card_limit_exhibit` a weak positive linear correlation.



### 1.4.3 Transactions

In [0]:
transactions_df.info()

In [0]:
transactions_df.describe(percentiles=[0.85, 0.95, 0.995])

In [0]:
transactions_df['event'].value_counts().plot.bar()
plt.xticks(rotation=0)
plt.show()

In [0]:
transactions_df.groupby('time_since_test_start')['account_id'].count().plot.line()
plt.show()

In [0]:
transactions_df.groupby(transactions_df['time_since_test_start'])['amount'].sum().plot.line()
plt.show()

In [0]:
transactions_df.groupby(transactions_df['time_since_test_start'])['reward'].sum().plot.line()
plt.show()

In [0]:
transactions_df.amount.sum()

In [0]:
transactions_df.reward.sum()

In [0]:
transactions_df.event.value_counts()

**Conclusions:**
- The experiment lasted 30 days.

- The average transaction `amount` was R$12.77.

- A total of 76277 offers were sent; 75.67% were viewed, and 44% were completed.

- There appears to be a seasonal pattern associated with the `offer received` event.

- Transaction `amounts` show a positive upward trend over time since the start of the test.

- The total `reward` sum also exhibits a periodic upward trend throughout the test period.

- Unfortunately, it is not possible to draw causal conclusions from the available data.

### 1.4.4 Combined Data

In [0]:
combined_df = transactions_df.merge(profile_df, left_on='account_id', right_on='id', how='left', validate='many_to_one').drop(columns='id')
combined_df.dropna(subset=['age', 'credit_card_limit', 'gender', 'registered_on'], inplace=True)
combined_df.info()

In [0]:
combined_df = combined_df.merge(offers_df, left_on='offer_id_unified', right_on='id', how='left').drop(columns=['id'])
combined_df

In [0]:
combined_df.describe()

In [0]:
combined_df[combined_df['event'] == 'offer received'].offer_id_unified.value_counts()

In [0]:
counts = combined_df[combined_df['event'] == 'offer received'] \
            .offer_id_unified.value_counts()
counts.plot(kind='bar')
plt.xticks([], [])
plt.show()

In [0]:
age_mean = combined_df.groupby("age")["amount"].mean()
plt.scatter(age_mean.index, age_mean.values, s=10)
plt.show()

In [0]:
age_sum = combined_df.groupby("age")["amount"].sum()
plt.scatter(age_sum.index, age_sum.values, s=10)
plt.show()

In [0]:
cc_mean = combined_df.groupby("credit_card_limit")["amount"].mean()
plt.scatter(cc_mean.index, cc_mean.values, s=10)
plt.show()

In [0]:
num_variables = ['age', 'credit_card_limit', 'amount']
correlation = combined_df[num_variables].corr(method="spearman")
print(correlation)

In [0]:
correlation = combined_df[num_variables].corr(method="pearson")
print(correlation)

In [0]:
sns.catplot(
    data=combined_df[combined_df['event'] != 'transaction'],
    x='offer_type',
    kind='count',
    hue='event',
    aspect=1.5
)
plt.show()

In [0]:
combined_df.groupby('offer_type').event.value_counts()

In [0]:
ids_with_offer = combined_df.loc[
    combined_df["event"] == "offer received", 
    "account_id"
].unique()

df_users_without_offer = combined_df[
    ~combined_df["account_id"].isin(ids_with_offer)
]

print(f"Number of users who did not receive any offer: {combined_df.account_id.nunique() - len(ids_with_offer)}")
df_users_without_offer

**Conclusions:**

- Each offer was received by approximately 6,600 customers. This is an important detail because the simulated data does not introduce bias toward any specific offer. Therefore every customer has an equal chance of receiving any offer.

- The total spending volume by age reveals a pronounced peak between 50 and 65 years old, indicating that this age group either contains more customers or performs more transactions. Younger and older age groups show lower spending volume, suggesting less representation in the dataset.

- There is a moderate Spearman correlation between transaction `amount` and `credit_card_limit`, meaning the relationship is monotonic but not necessarily linear.

- The `informational` offer type does not generate `offer completed` events, so its effectiveness must be evaluated using alternative metrics.

- The `bogo` offer type shows the highest volume of viewed events.

- There are users in the dataset who did not receive any offer throughout the experiment

## 1.5 Data Split

Divide the cleaned dataset into training and testing sets.

In [0]:
combined_df = combined_df[
    combined_df["account_id"].isin(ids_with_offer)
]
print('Shape: ', combined_df.shape)

df_users_without_offer = profile_df[
    ~profile_df["id"].isin(ids_with_offer)
]
df_users_without_offer.to_csv(f'{path}/users_without_offers.csv', index=False)
df_users_without_offer

In [0]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

for train_idx, test_idx in gss.split(combined_df, groups=combined_df["account_id"]):
    train_df = combined_df.iloc[train_idx]
    test_df  = combined_df.iloc[test_idx]

In [0]:
print(f"Combined Info - Shape: {combined_df.shape} \nAccounts: {combined_df['account_id'].nunique()} \nEvents: {combined_df['event'].value_counts(normalize=True)} \n")

print(f"Train Info - Shape: {train_df.shape} \nAccounts: {train_df['account_id'].nunique()} \nEvents: {train_df['event'].value_counts(normalize=True)}\n")

print(f"Test Info - Shape: {test_df.shape} \nAccounts: {test_df['account_id'].nunique()} \nEvents: {test_df['event'].value_counts(normalize=True) }\n")

In [0]:

train_df.to_csv(f'{path}/train.csv', index=False)
test_df.to_csv(f'{path}/test.csv', index=False)

**Conclusions:**

- Users who did not receive any offers were removed from the main dataset and saved separately for potential future demonstrations of the model.

- The data was split by user to ensure that the same individuals do not appear in both the training and testing sets.